## DIANNA demo with Imagenet

This notebook showcases the use of DIANNA on a Resnet network trained on a subset of the Imagenet dataset, containing 1000 categories.

#### Install and import packages

In [ ]:
!pip install git+https://github.com/dianna-ai/dianna.git

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing import image

np.random.seed(42)

In [ ]:
# for google colab: clone and cd into the repository directory
try:
    import google.colab
    colab = True
except ImportError:
    colab = False
    
if colab:
    ! git clone https://github.com/loostrum/dianna_demo.git
    os.chdir('dianna_demo')

data_path = 'data/bee.jpg'

#### Define model and helper functions

In [ ]:
class Model():
    def __init__(self):
        K.set_learning_phase(0)
        self.model = ResNet50()
        self.input_size = (224, 224)
        
    def run_on_batch(self, x):
        return self.model.predict(x)
    
model = Model()

In [ ]:
def load_img(path):
    img = image.load_img(path, target_size=model.input_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x

In [ ]:
def class_name(idx):
    return decode_predictions(np.eye(1, 1000, idx))[0][0][1]

#### Run model on example data

In [ ]:
img, scaled_img = load_img(data_path)
plt.imshow(img)
plt.axis('off')

In [ ]:
sorted_classes = np.argsort(model.model.predict([scaled_img]))[0][::-1]
print('\n'.join(['{:4} {}'.format(c, class_name(c)) for c in sorted_classes[:10]]))

## Explainable AI with DIANNA

In [ ]:
import dianna

In [ ]:
heatmaps = dianna.explain_image(model.run_on_batch, scaled_img, method='RISE')

For each class (1000 in this case), an explanation with a shape equal to the input image is generated.

In [ ]:
print(heatmaps.shape)

The explanations can be considered images and can be plotted with e.g. `matplotlib`.  
DIANNA also includes visualization tools.

In [ ]:
from dianna.visualization import plot_image

In [ ]:
for current_class_id in sorted_classes[:5]:
    plot_image(heatmaps[current_class_id], original_data=image.img_to_array(img)/255., heatmap_cmap='jet', show_plot=False)
    plt.title(f'Explanation for `{class_name(current_class_id)}`')
    plt.axis('off')

Note that in these plots, red means important and blue means unimportant.